In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, LongType, MapType, Row
from pyspark.sql.functions import *
from pyspark.sql import Window
import numpy as np
import pandas as pd
import sys

spark = SparkSession.builder.appName("Auto Report–Spark").getOrCreate()



21/10/30 18:53:45 WARN Utils: Your hostname, Mallorys-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.66 instead (on interface en0)
21/10/30 18:53:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/30 18:53:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/10/30 18:53:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/10/30 18:53:47 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [14]:
from pipe import chain

In [2]:
schema = StructType([ \
    StructField('incident_id', IntegerType(), True), 
    StructField('incident_type', StringType(), True),
    StructField('vin_num', StringType(), True),
    StructField('make', StringType(), True),
    StructField('model', StringType(), True),
    StructField('year', StringType(), True),
    StructField('incident_date', DateType(), True),
    StructField('desc', StringType(), True)
])


In [4]:
df = spark.read.format("csv") \
    .option("header", False) \
    .schema(schema) \
    .load("/Users/mallory/Desktop/DataEngineering/Springboard/DistComp/hadoop_auto/data.csv")
df.show()

+-----------+-------------+-----------------+--------+------+----+-------------+--------------------+
|incident_id|incident_type|          vin_num|    make| model|year|incident_date|                desc|
+-----------+-------------+-----------------+--------+------+----+-------------+--------------------+
|          1|            I|VXIO456XLBB630221|  Nissan|Altima|2003|   2002-05-08|Initial sales fro...|
|          2|            I|INU45KIOOPA343980|Mercedes|  C300|2015|   2014-01-01|Sold from EuroMotors|
|          3|            A|VXIO456XLBB630221|    null|  null|null|   2014-07-02|   Head on collision|
|          4|            R|VXIO456XLBB630221|    null|  null|null|   2014-08-05| Repair transmission|
|          5|            I|VOME254OOXW344325|Mercedes|  E350|2015|   2014-02-01|    Sold from Carmax|
|          6|            R|VOME254OOXW344325|    null|  null|null|   2015-02-06|Wheel allignment ...|
|          7|            R|VXIO456XLBB630221|    null|  null|null|   2015-01-01|Re

In [5]:
def extract_key_vin_value(x):
    """:param x: data source loaded into SparkSession,
        :output: dictionary tuple with mapping values to be transformed into MapType"""

    vin_number = x.vin_num
    make = x.make
    year = x.year
    model = x.model
  
    return (vin_number, {"make": make, "year": year, "model": model})


In [6]:
vin_kv = df.rdd.map(lambda x: extract_key_vin_value(x))


In [7]:
# creating a bridge table to collect master information for each make and model
bridge_schema = StructType([
    StructField("vin_key", StringType(), True),
    # use MapType to make use of key-value pairs returned by function
    StructField("properties", MapType(StringType(), StringType(), True))
])

bridge_df = spark.createDataFrame(data=vin_kv, schema=bridge_schema)
bridge_df.show(truncate=False)
# check schema
bridge_df.printSchema()


+-----------------+------------------------------------------------+
|vin_key          |properties                                      |
+-----------------+------------------------------------------------+
|VXIO456XLBB630221|{model -> Altima, year -> 2003, make -> Nissan} |
|INU45KIOOPA343980|{model -> C300, year -> 2015, make -> Mercedes} |
|VXIO456XLBB630221|{model -> null, year -> null, make -> null}     |
|VXIO456XLBB630221|{model -> null, year -> null, make -> null}     |
|VOME254OOXW344325|{model -> E350, year -> 2015, make -> Mercedes} |
|VOME254OOXW344325|{model -> null, year -> null, make -> null}     |
|VXIO456XLBB630221|{model -> null, year -> null, make -> null}     |
|EXOA00341AB123456|{model -> SL550, year -> 2016, make -> Mercedes}|
|VOME254OOXW344325|{model -> null, year -> null, make -> null}     |
|VOME254OOXW344325|{model -> null, year -> null, make -> null}     |
|EXOA00341AB123456|{model -> null, year -> null, make -> null}     |
|EXOA00341AB123456|{model -> null,

In [8]:
# options here were map_concat(), coalesce(), and explode()
# explode map column to create a new row for each element in the given map column
bridge_df = bridge_df.select("vin_key", explode("properties"))

bridge_df.printSchema()  # schema will validate the explosion
bridge_df.show()


root
 |-- vin_key: string (nullable = true)
 |-- key: string (nullable = false)
 |-- value: string (nullable = true)

+-----------------+-----+--------+
|          vin_key|  key|   value|
+-----------------+-----+--------+
|VXIO456XLBB630221|model|  Altima|
|VXIO456XLBB630221| year|    2003|
|VXIO456XLBB630221| make|  Nissan|
|INU45KIOOPA343980|model|    C300|
|INU45KIOOPA343980| year|    2015|
|INU45KIOOPA343980| make|Mercedes|
|VXIO456XLBB630221|model|    null|
|VXIO456XLBB630221| year|    null|
|VXIO456XLBB630221| make|    null|
|VXIO456XLBB630221|model|    null|
|VXIO456XLBB630221| year|    null|
|VXIO456XLBB630221| make|    null|
|VOME254OOXW344325|model|    E350|
|VOME254OOXW344325| year|    2015|
|VOME254OOXW344325| make|Mercedes|
|VOME254OOXW344325|model|    null|
|VOME254OOXW344325| year|    null|
|VOME254OOXW344325| make|    null|
|VXIO456XLBB630221|model|    null|
|VXIO456XLBB630221| year|    null|
+-----------------+-----+--------+
only showing top 20 rows



In [9]:
# get distinct values & drop null to avoid duplication
df_distinct = bridge_df.select("vin_key", "key", "value").distinct().na.drop()
df_distinct.show(truncate=False)


+-----------------+-----+--------+
|vin_key          |key  |value   |
+-----------------+-----+--------+
|UXIA769ABCC447906|make |Toyota  |
|INU45KIOOPA343980|year |2015    |
|VXIO456XLBB630221|model|Altima  |
|EXOA00341AB123456|year |2016    |
|VOME254OOXW344325|year |2015    |
|UXIA769ABCC447906|year |2017    |
|UXIA769ABCC447906|model|Camery  |
|VOME254OOXW344325|model|E350    |
|EXOA00341AB123456|model|SL550   |
|VOME254OOXW344325|make |Mercedes|
|VXIO456XLBB630221|make |Nissan  |
|INU45KIOOPA343980|make |Mercedes|
|INU45KIOOPA343980|model|C300    |
|VXIO456XLBB630221|year |2003    |
|EXOA00341AB123456|make |Mercedes|
+-----------------+-----+--------+



In [19]:
# get rid of null values to get year make and model
map_df = df_distinct.select("vin_key", create_map("key", "value").alias("map")) \
    .groupBy("vin_key") \
    .agg(collect_list("map").alias("year_make_model"))

map_df.show(truncate=False)


+-----------------+------------------------------------------------------+
|vin_key          |year_make_model                                       |
+-----------------+------------------------------------------------------+
|VOME254OOXW344325|[{year -> 2015}, {model -> E350}, {make -> Mercedes}] |
|UXIA769ABCC447906|[{make -> Toyota}, {year -> 2017}, {model -> Camery}] |
|VXIO456XLBB630221|[{model -> Altima}, {make -> Nissan}, {year -> 2003}] |
|INU45KIOOPA343980|[{year -> 2015}, {make -> Mercedes}, {model -> C300}] |
|EXOA00341AB123456|[{year -> 2016}, {model -> SL550}, {make -> Mercedes}]|
+-----------------+------------------------------------------------------+



In [28]:
map_df.printSchema()

root
 |-- vin_key: string (nullable = true)
 |-- year_make_model: array (nullable = false)
 |    |-- element: map (containsNull = false)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)



In [34]:
# df2 = df.withColumn("languagesAtSchool",
#                     concat_ws(",", col("languagesAtSchool")))
df = map_df.withColumn("year_make_model", concat_ws(",", col("year_make_model")))

AnalysisException: cannot resolve 'concat_ws(',', year_make_model)' due to data type mismatch: argument 2 requires (array<string> or string) type, however, 'year_make_model' is of array<map<string,string>> type.;
'Project [vin_key#122, concat_ws(,, year_make_model#240) AS year_make_model#267]
+- Aggregate [vin_key#122], [vin_key#122, collect_list(map#234, 0, 0) AS year_make_model#240]
   +- Project [vin_key#122, map(key#135, value#136) AS map#234]
      +- Filter atleastnnonnulls(3, vin_key#122, key#135, value#136)
         +- Deduplicate [vin_key#122, key#135, value#136]
            +- Project [vin_key#122, key#135, value#136]
               +- Project [vin_key#122, key#135, value#136]
                  +- Generate explode(properties#123), false, [key#135, value#136]
                     +- LogicalRDD [vin_key#122, properties#123], false


In [33]:
# remove map from list format?
map_df.select(element_at(map_df.year_make_model, lit("year")))

AnalysisException: cannot resolve 'element_at(year_make_model, 'year')' due to data type mismatch: Input to function element_at should have been array followed by a int, but it's [array<map<string,string>>, string].;
'Project [unresolvedalias(element_at(year_make_model#240, year, false), Some(org.apache.spark.sql.Column$$Lambda$3471/0x0000000801135040@397ddc84))]
+- Aggregate [vin_key#122], [vin_key#122, collect_list(map#234, 0, 0) AS year_make_model#240]
   +- Project [vin_key#122, map(key#135, value#136) AS map#234]
      +- Filter atleastnnonnulls(3, vin_key#122, key#135, value#136)
         +- Deduplicate [vin_key#122, key#135, value#136]
            +- Project [vin_key#122, key#135, value#136]
               +- Project [vin_key#122, key#135, value#136]
                  +- Generate explode(properties#123), false, [key#135, value#136]
                     +- LogicalRDD [vin_key#122, properties#123], false


In [31]:
map_df.withColumn("year", map_df.year_make_model.get_item("year")) \
    .withColumn("make", map_df.year_make_model.get_item("model")) \
    .withColumn("model", map_df.year_make_model.get_item("year"))


In [20]:
vin_kv.collect()

[('VXIO456XLBB630221', {'make': 'Nissan', 'year': '2003', 'model': 'Altima'}),
 ('INU45KIOOPA343980', {'make': 'Mercedes', 'year': '2015', 'model': 'C300'}),
 ('VXIO456XLBB630221', {'make': None, 'year': None, 'model': None}),
 ('VXIO456XLBB630221', {'make': None, 'year': None, 'model': None}),
 ('VOME254OOXW344325', {'make': 'Mercedes', 'year': '2015', 'model': 'E350'}),
 ('VOME254OOXW344325', {'make': None, 'year': None, 'model': None}),
 ('VXIO456XLBB630221', {'make': None, 'year': None, 'model': None}),
 ('EXOA00341AB123456', {'make': 'Mercedes', 'year': '2016', 'model': 'SL550'}),
 ('VOME254OOXW344325', {'make': None, 'year': None, 'model': None}),
 ('VOME254OOXW344325', {'make': None, 'year': None, 'model': None}),
 ('EXOA00341AB123456', {'make': None, 'year': None, 'model': None}),
 ('EXOA00341AB123456', {'make': None, 'year': None, 'model': None}),
 ('VOME254OOXW344325', {'make': None, 'year': None, 'model': None}),
 ('UXIA769ABCC447906', {'make': 'Toyota', 'year': '2017', 'mod

In [22]:
def populate_make(df_with_nulls, map=map_df):
    """
    Populate empty make values in dataset using map_df.
    :param map: (optional) dataframe with mapped values to fill nulls
    :param data_with_nulls: Spark rdd with null values to be replaced
    :output: dataframe with filled null values 
    """

    # creating a bridge table to collect master information for each make and model
    bridge_schema = StructType([
        StructField("vin_key", StringType(), True),
        # use MapType to make use of key-value pairs returned by function
        StructField("properties", MapType(StringType(), StringType(), True))
    ])

    bridge_df = spark.createDataFrame(data=vin_kv, schema=bridge_schema)

    # explode map column to create a new row for each element in the given map column
    bridge_df = bridge_df.select("vin_key", explode("properties"))

    # get distinct values & drop null to avoid duplication
    df_distinct = bridge_df.select("vin_key", "key", "value").distinct().na.drop()

    # get rid of null values to get year make and model
    map_df = df_distinct.select("vin_key", create_map("key", "value").alias("map")) \
        .groupBy("vin_key") \
        .agg(collect_list("map").alias("year_make_model"))
    
    

    make = 
    model =
    year = 

    return # values


In [ ]:
populate_make()

In [ ]:
enhance_make = vin_kv.groupByKey().flatMap(lambda kv: populate_make(kv[1]))